<a href="https://colab.research.google.com/github/MCygert/preparing_for_tensorflow_certificate/blob/main/04_transfer_learning_in_tensorflow_part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer learning
Using pretrained model for you own benefit.

In [1]:
!nvidia-smi

Sat Jan  8 22:02:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Download and become one with data

In [2]:
# Get data (10$ of 10 food classes)
import zipfile
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip")
zip_ref.extractall()
zip_ref.close()

--2022-01-08 22:02:28--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.216.128, 173.194.218.128, 64.233.170.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.216.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M   225MB/s    in 0.7s    

2022-01-08 22:02:29 (225 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [3]:
import os
for dirpath, dirnames, filenames in os.walk("10_food_classes_10_percent"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} iamges in '{dirpath}")

There are 2 directories and 0 iamges in '10_food_classes_10_percent
There are 10 directories and 0 iamges in '10_food_classes_10_percent/train
There are 0 directories and 75 iamges in '10_food_classes_10_percent/train/grilled_salmon
There are 0 directories and 75 iamges in '10_food_classes_10_percent/train/pizza
There are 0 directories and 75 iamges in '10_food_classes_10_percent/train/hamburger
There are 0 directories and 75 iamges in '10_food_classes_10_percent/train/chicken_curry
There are 0 directories and 75 iamges in '10_food_classes_10_percent/train/ice_cream
There are 0 directories and 75 iamges in '10_food_classes_10_percent/train/fried_rice
There are 0 directories and 75 iamges in '10_food_classes_10_percent/train/ramen
There are 0 directories and 75 iamges in '10_food_classes_10_percent/train/chicken_wings
There are 0 directories and 75 iamges in '10_food_classes_10_percent/train/sushi
There are 0 directories and 75 iamges in '10_food_classes_10_percent/train/steak
There are

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
IMAGE_SHAPE = (224, 224)
BATCH_SIZE = 32

train_dir= "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"
train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

train_data_10_percent = train_datagen.flow_from_directory(train_dir, target_size = IMAGE_SHAPE,
                                                          batch_size=BATCH_SIZE,
                                                          class_mode='categorical')

test_data_10_percent = test_datagen.flow_from_directory(test_dir, target_size = IMAGE_SHAPE,
                                                          batch_size=BATCH_SIZE,
                                                          class_mode='categorical')


Found 750 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


## Setting up callbacks (things to run whilst our model trains)

Extra functionality which I can add to doy while training or after. 
* Tracking experiments with the TensorBoard callback
* Model checkpointing
* Stopping a model from training (before it trains too long) EarlyStopping callback

In [5]:
import datetime
import tensorflow as tf

def create_tensorboard_callback(dir_name, experiment_name):
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%D-%H%")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir
  )
  print(f"Saving tensorboard log files to: {log_dir}")
  return tensorboard_callback

## Creating tensorflow models using tensorflow hub


In [6]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers


In [7]:
# Resnet 50 V2 feature vector
resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"

# Original: EfficientNetB0 feature vector (version 1)
# efficientnet_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

# # New: EfficientNetB0 feature vector (version 2)
efficientnet_url = "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b0/feature_vector/2"

In [8]:
def create_model(model_url, num_classes=10):
  feature_extractor_layer = hub.KerasLayer(model_url,
                                           trainable=False,
                                           name="feature_extraction_layer",
                                           input_shape=IMAGE_SHAPE + (3,))
  model = tf.keras.Sequential([
                               feature_extractor_layer,
                               layers.Dense(num_classes, activation="softmax", name="output_layer")
  ])
  return model

In [9]:
# Create Resnet model
resnet_model = create_model(resnet_url)

resnet_model.compile(loss='categorical_crossentropy',
                    optimizer=tf.keras.optimizers.Adam(),
                    metrics=['accuracy'])
resnet_history = resnet_model.fit(train_data_10_percent,
                                  steps_per_epoch=len(train_data_10_percent),
                                  epochs=5,
                                  validation_data=test_data_10_percent,
                                  validation_steps=len(test_data),
                                  callbacks=[create_tensorboard_callback("tensorflow_hub", "resnet50V2")])

Saving tensorboard log files to: tensorflow_hub/resnet50V2/20220101/08/22-22%
Epoch 1/5
24/24 [==============================] - 28s 690ms/step - loss: 1.9743 - accuracy: 0.3520 - val_loss: 1.2602 - val_accuracy: 0.6140
Epoch 2/5
24/24 [==============================] - 15s 650ms/step - loss: 0.8984 - accuracy: 0.7453 - val_loss: 0.8558 - val_accuracy: 0.7348
Epoch 3/5
24/24 [==============================] - 15s 641ms/step - loss: 0.6037 - accuracy: 0.8347 - val_loss: 0.7560 - val_accuracy: 0.7620
Epoch 4/5
24/24 [==============================] - 15s 647ms/step - loss: 0.4571 - accuracy: 0.8787 - val_loss: 0.7064 - val_accuracy: 0.7712
Epoch 5/5
24/24 [==============================] - 15s 649ms/step - loss: 0.3678 - accuracy: 0.9213 - val_loss: 0.6744 - val_accuracy: 0.7856
